## Make Example Plot

Making a plot that maps some numeric summary metric to sites in a protein. Plot is specified using `Altair` and saved as `JSON` to be embedded into an `HTML` document.

In [1]:
import altair as alt

import pandas as pd

import polyclonal

In [2]:
# allow more rows for Altair
_ = alt.data_transformers.disable_max_rows()

### Read in the example data

This example data is coming from a [DMS experiment of SARS-Cov-2 Spike](https://dms-vep.github.io/SARS-CoV-2_Omicron_BA.1_spike_DMS/fit_polyclonal_Lib-3_2022-06-22_thaw-1_268C.59A_1.html) performed by Bernadeta in the Bloom lab.

In [3]:
# Example data
prob_escape_file = "../data/Lib-3_2022-06-22_thaw-1_268C.59A_1_prob_escape.csv"
prob_escape = pd.read_csv(
    prob_escape_file,
    keep_default_na=False,
    na_values="nan"
).query("`no-antibody_count` >= no_antibody_count_threshold")
prob_escape.head()

,library,antibody_sample,no-antibody_sample,aa_substitutions_sequential,n_aa_substitutions,barcode,prob_escape,prob_escape_uncensored,antibody_count,no-antibody_count,antibody_neut_standard_count,no-antibody_neut_standard_count,total_no_antibody_count,no_antibody_count_threshold,aa_substitutions_reference,antibody,antibody_concentration
0,Lib-3,2022-06-22_thaw-1_antibody_268C.59A_20.2_1,2022-06-22_thaw-1_no-antibody_control_1,P26S E551V V619G K761S,4,CTTGAATAGGACATTC,0.6349,0.6349,19895,3502,2206727,246612,72545787,73,P26S E554V V622G K764S,268C.59A,20.2
1,Lib-3,2022-06-22_thaw-1_antibody_268C.59A_20.2_1,2022-06-22_thaw-1_no-antibody_control_1,E551K,1,ACAAATAAAGGCCTAT,0.4773,0.4773,15858,3713,2206727,246612,72545787,73,E554K,268C.59A,20.2
2,Lib-3,2022-06-22_thaw-1_antibody_268C.59A_20.2_1,2022-06-22_thaw-1_no-antibody_control_1,E551Q,1,GTAGTGAGTTTCTAAC,0.3557,0.3557,13272,4170,2206727,246612,72545787,73,E554Q,268C.59A,20.2
3,Lib-3,2022-06-22_thaw-1_antibody_268C.59A_20.2_1,2022-06-22_thaw-1_no-antibody_control_1,E551K,1,TACATATAAGGGACAA,0.5298,0.5298,12093,2551,2206727,246612,72545787,73,E554K,268C.59A,20.2
4,Lib-3,2022-06-22_thaw-1_antibody_268C.59A_20.2_1,2022-06-22_thaw-1_no-antibody_control_1,K555R G766S D1115W,3,GGGCACCCTCATGCAC,0.5606,0.5606,11403,2273,2206727,246612,72545787,73,K558R G769S D1118W,268C.59A,20.2


### Configure the analysis

Using the same configuration parameters from Jesse and Bernadeta's analysis. 

In [4]:
# Antibody name
antibody = prob_escape["antibody"].unique()

# Reference site numbering 
reference_sites = (
    pd.read_csv("../data/site_numbering_map.csv")
    .sort_values("sequential_site")["reference_site"]
    .tolist()
)

# Antibody specific configuration parameters for polyclonal
antibody_config = {
    'max_epitopes': 1,
    'n_bootstrap_samples': 50,
    'reg_escape_weight': 0.1,
    'reg_spread_weight': 0.25,
    'reg_activity_weight': 1.0,
    'times_seen': 3,
    'min_epitope_activity_to_include': 0.2
}


print(f"Analysis for antibody: {antibody[0]}\n")
print("Configuration Parameters:" + ''.join([f'\n\t{key}: {value}' for key, value in antibody_config.items()]))

Analysis for antibody: 268C.59A

Configuration Parameters:
	max_epitopes: 1
	n_bootstrap_samples: 50
	reg_escape_weight: 0.1
	reg_spread_weight: 0.25
	reg_activity_weight: 1.0
	times_seen: 3
	min_epitope_activity_to_include: 0.2


### Fit the `polyclonal` model 

Fitting `polyclonal` model to get the escape scores for each variant. 

In [5]:
## == Parameters for fitting the model == ##
times_seen = antibody_config["times_seen"]

max_epitopes = antibody_config["max_epitopes"]

fit_kwargs = {
    "reg_escape_weight": antibody_config["reg_escape_weight"],
    "reg_spread_weight": antibody_config["reg_spread_weight"],
    "reg_activity_weight": antibody_config["reg_activity_weight"],
}

min_epitope_activity_to_include = antibody_config["min_epitope_activity_to_include"]

## == Fit and choose the best model == ##

models = []

for n_epitopes in range(1, max_epitopes + 1):
    print(f"\nFitting model with {n_epitopes=}")

    # create model
    model = polyclonal.Polyclonal(
        n_epitopes=n_epitopes,
        data_to_fit=prob_escape.rename(
            columns={
                "antibody_concentration": "concentration",
                "aa_substitutions_reference": "aa_substitutions",
            }
        ),
        alphabet=polyclonal.AAS_WITHSTOP_WITHGAP,
        sites=reference_sites,
    )

    # fit model
    opt_res = model.fit(logfreq=200, **fit_kwargs)

    # display activities
    print("Activities of epitopes:")
    display(model.activity_wt_df.round(1))
    print("Max and mean absolute-value escape at each epitope:")
    display(
        model.mut_escape_df.groupby("epitope")
        .aggregate(
            max_escape=pd.NamedAgg("escape", "max"),
            mean_abs_escape=pd.NamedAgg("escape", lambda s: s.abs().mean()),
        )
        .round(1)
    )

    # stop if activity below threshold for any epitope and fit at least one epitope
    if len(models) and any(
        model.activity_wt_df["activity"] <= min_epitope_activity_to_include
    ):
        print(f"Stop fitting, epitope has activity <={min_epitope_activity_to_include}")
        models.append(model)
        model = models[-2]  # get previous model
        break
    else:
        models.append(model)

print(f"\nThe selected model has {len(model.epitopes)} epitopes")


Fitting model with n_epitopes=1
# First fitting site-level model.
# Starting optimization of 1200 parameters at Fri Aug 19 11:42:26 2022.
         step     time_sec         loss     fit_loss   reg_escape   reg_spread reg_activity
            0     0.085136       1230.7       1229.8            0            0      0.90499
          164       12.868       1144.8       1131.2       12.449            0       1.0918
# Successfully finished at Fri Aug 19 11:42:39 2022.
# Starting optimization of 7126 parameters at Fri Aug 19 11:42:39 2022.
         step     time_sec         loss     fit_loss   reg_escape   reg_spread reg_activity
            0      0.11135       1799.9         1720       78.847   3.9437e-31       1.0918
          135       16.063       1731.9       1696.3       29.297       5.0499       1.2108
# Successfully finished at Fri Aug 19 11:42:55 2022.
Activities of epitopes:


,epitope,activity
0,1,1.3


Max and mean absolute-value escape at each epitope:


,max_escape,mean_abs_escape
epitope,,
1,5.3,0.1



The selected model has 1 epitopes


### Plotting the escape values

Now, I'll plot the results of the fit model using `Altair`. These are the example plots that I'll connect to the protein strucutre. 

In [6]:
example_chart = model.mut_escape_lineplot(min_times_seen=times_seen, height=175, width=600, zoom_bar_width=600)
example_chart

alt.VConcatChart(...)

To get save the plot as compiled `Vega` and integrate into the website via `vegaEmbed`, click on the `...` in the upper left of the plot and select **View Compiled Vega** and save the page as `example.chart.json` in `src/static/`.